# 0 - Setup & imports

In [14]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Add project root
project_root = Path("..").resolve()
sys.path.append(str(project_root))
from pathlib import Path





# Context

## Why an EDS alumnus made this workshop

With the rapid adoption of AI tools, two professional profiles are increasingly valuable.

The first is the extreme specialist or researcher:

- Works at the frontier of knowledge

- Solves problems where AI lacks sufficient depth or domain understanding

- Pushes into highly technical, unresolved areas

The second is the generalist — the role you are training for in this course:

- Does not need to know every concept in detail

- Needs to know what exists and how systems connect

- Uses AI as a tool to explore, clarify, and reason through unfamiliar components

In practice, success as a generalist means:

- Understanding the full pipeline of a problem

- Asking the right questions

- Iterating with AI to deepen understanding where needed, on the job

## Job Market

![job_description](../images/job_description.png)


## Real pipeline

![court](../images/court.png)


## It's not you, it's everybody 

![container](../images/container.png)


# Full Pipeline 

- Problem framing : Define the business question, target variable, constraints, and success metric.

- Data collection : Pull data from databases, APIs, logs, files, or third-party sources.

- Data understanding & EDA : Inspect distributions, missing values, bias, leakage, and sanity-check assumptions.

- Data cleaning & feature engineering : Handle missing values, encode variables, create meaningful features, align schemas.

- Train / validation / test split : Split data in a way that matches production reality (time-based if needed).

- Modeling : Train baseline → advanced models, tune hyperparameters, compare approaches.

- Evaluation : Use the right metrics (ROC-AUC, F1, precision/recall, etc.), error analysis.

- Experiment tracking : Log parameters, metrics, artifacts (e.g., with MLflow).

- Packaging : Save the model, dependencies, and environment (often via Docker).

- Deployment : Expose the model via API, batch job, or app (cloud or on-prem).

- Monitoring & retraining : Track performance, drift, failures; retrain when data or behavior changes.

# Building an Early-Warning System for Riverbend School District

![High school](../images/high_school.jpg)


# 1 - Scoping Phase — Understanding the Human Problem

As a data science consultant, you are responsible for helping the Riverbend School District address a growing concern around student disengagement and dropout risk. You will work directly with Dr. Aris Thorne, the district superintendent and your primary point of contact, to understand how the district defines success, risk, and failure. Your task is to translate this institutional knowledge into a complete, end-to-end data science solution — from problem scoping and data exploration to modeling, evaluation, and deployment — resulting in an early-warning system that can be used by district leadership to intervene before it is too late.

## 📖 The Situation on the Ground


The Riverbend School District has experienced growing concern about student dropout and has asked an external data science consultancy team to investigate.

Historically, the district operated under a traditional educational model:

- In-person learning with strict physical attendance requirements

- Grades recorded on a 5-point scale

Under this system, district leadership believed that:

- Physical absence was a strong warning signal

- Declining grades reliably indicated academic risk

- Student disengagement was relatively easy to detect

Despite this, the district has observed an increase in dropout cases and growing uncertainty about when and how students disengage. As a result, leadership has turned to data science students to help design an early-warning system capable of identifying at-risk students before intervention becomes impossible.

## 🔄 Project Lumina: A System in Transition

Beginning in Semester 3, Riverbend launched Project Lumina, a district-wide reform that fundamentally altered how learning takes place.

Under this new model:

- Students can attend class physically or digitally, video lectures replace some in-person sessions, and academic engagement is no longer tied to daily physical presence

- Grades are now recorded on a 20-point international scale

From an administrative perspective, the system still looks the same — students, courses, grades, attendance. But behavior no longer means what it used to mean.

## 🎯 Objective


- The district has hired you to design and deliver an early-warning system that identifies students at risk of dropping out early enough to enable meaningful intervention.

## ⚠️ Important Constraint

It is a real life project. Thus, at the start of the project:

- There is no predefined target variable

- There is no data dictionary

- There is no explicit definition of “dropout” in the data


In practice:

- Models are not built first

- Understanding the institution comes before modeling

## 🧠 Key Questions Before Writing Code

Before touching the data, you must answer:

- How does the district define a dropout? Has this definition changed over time?

- Which behaviors stopped meaning the same thing after Project Lumina?

- What events could affect your model?

If these questions are not addressed:

- The model may perform well on paper but fail in real-world deployment

Ask questions to Dr.Thorne, the superintendent : https://riverbend.streamlit.app/



# 2 - Pull Data

- In real-world data science projects, data is rarely provided as clean CSV files. Instead, it often lives inside operational systems such as databases, internal tools, APIs, or even collections of PDFs. Accessing this data is part of the problem-solving process.

- For Riverbend School District, student data is stored in a PostgreSQL database. We will use psycopg2, a standard Python adapter for PostgreSQL, to connect directly to the database and execute SQL queries. Psycopg2 is widely used in production because it is reliable and closely aligned with PostgreSQL’s native behavior.

- Query results will then be loaded into pandas DataFrames, allowing us to combine SQL-based data extraction with Python-based exploration and modeling.

It is good practice to ask for the database schema. Some databases can be incredibly complex, and there is, more often than not, no data dictionary.
This is usually a second part of the **scoping phase* , in which you choose the variables that you will pull. You need to ask lots of questions about the problem, the metrics you will need, and about the variables as well. 


![schema](../images/schema.png)


Before diving into code, even at the exploration phase, we want to have a look at our problem, our schema, and ask ourself a few questions.

- Based on our objective, what metrics do we want to use? Accuracy?

- Which variables appear informative but are now misleading if used naïvely?

- What drifts can influence the model?

- Does data imbalance matter?

## Example pull

Here is a simple example with the students table. You will recognize SQL syntax.

In [ ]:
import psycopg2
import pandas as pd

# Connect to Supabase Postgres
conn = psycopg2.connect(
    host="aws-1-us-west-1.pooler.supabase.com",
    port=5432,
    database="postgres",
    user="postgres.zzeuaztvmbhfednagzlz",
    password="edsdataset2026",
    sslmode="require"
)

# Simple test query with the students tables
df = pd.read_sql(
    "SELECT * FROM students LIMIT 5;",
    conn
)

print(df)


/var/folders/k7/b0_b7t6j6n72t68sh4s7t8400000gn/T/ipykernel_7653/2445650149.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


  student_id  birth_year  gender race_ethnicity parent_income_bracket  \
0     S00000        2009    male          white                   low   
1     S00001        2010    male          white          upper_middle   
2     S00002        2008  female          white                   low   
3     S00003        2010    male          black          lower_middle   
4     S00004        2010    male          white                  high   

  parent_education_level  household_size primary_language  special_needs_flag  \
0                     hs               6          english               False   
1                  no_hs               6          english               False   
2               graduate               5            other               False   
3                     hs               4          english               False   
4                college               6          english               False   

  school_id  
0    SCH_05  
1    SCH_00  
2    SCH_01  
3    SCH_03  
4   

## Try to pull data yourself, after understanding variables and knowing what you want to pull, what joins you want to make etc.

**Note:** In practice, it is rare to pull entire database tables into a local environment. This step is included here for transparency and exploration. In real-world projects, data scientists typically prepare SQL queries that extract only the specific columns, time periods, and records needed for the task at hand, both for efficiency and data governance reasons.

In [ ]:
import psycopg2
import pandas as pd
import os

# ---------------------------------------------------------
# CONFIG
# ---------------------------------------------------------
OUTPUT_DIR = "../data/csv_files"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------------------------------------------------------
# CONNECT TO SUPABASE POSTGRES
# ---------------------------------------------------------

conn = psycopg2.connect(
    host="aws-1-us-west-1.pooler.supabase.com",
    port=5432,
    database="postgres",
    user="postgres.zzeuaztvmbhfednagzlz",
    password="edsdataset2026",
    sslmode="require"
)

# ---------------------------------------------------------
# TABLES TO BACK UP
# ---------------------------------------------------------

tables = [
    "students",
    "schools",
    "courses",
    "course_grades",
    "attendance_records",
    "learning_activity",
    "disciplinary_events"
]

# ---------------------------------------------------------
# PULL + SAVE EACH TABLE
# ---------------------------------------------------------

for table in tables:
    print(f"Pulling table: {table}")
    
    query = f"SELECT * FROM {table};"
    df = pd.read_sql(query, conn)
    
    output_path = os.path.join(OUTPUT_DIR, f"{table}.csv")
    df.to_csv(output_path, index=False)
    
    print(f"Saved {len(df)} rows to {output_path}")

# ---------------------------------------------------------
# CLEAN UP
# ---------------------------------------------------------

conn.close()
print("✅ All tables successfully backed up to CSV.")


Pulling table: students


/var/folders/k7/b0_b7t6j6n72t68sh4s7t8400000gn/T/ipykernel_7653/3659775494.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Saved 5950 rows to ../data/csv_backup/students.csv
Pulling table: schools


/var/folders/k7/b0_b7t6j6n72t68sh4s7t8400000gn/T/ipykernel_7653/3659775494.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Saved 10 rows to ../data/csv_backup/schools.csv
Pulling table: courses
Saved 4 rows to ../data/csv_backup/courses.csv
Pulling table: course_grades


...

/var/folders/k7/b0_b7t6j6n72t68sh4s7t8400000gn/T/ipykernel_7653/3659775494.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Saved 275080 rows to ../data/csv_backup/attendance_records.csv
Pulling table: learning_activity


/var/folders/k7/b0_b7t6j6n72t68sh4s7t8400000gn/T/ipykernel_7653/3659775494.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Saved 17534 rows to ../data/csv_backup/learning_activity.csv
Pulling table: disciplinary_events
Saved 134 rows to ../data/csv_backup/disciplinary_events.csv
✅ All tables successfully backed up to CSV.


/var/folders/k7/b0_b7t6j6n72t68sh4s7t8400000gn/T/ipykernel_7653/3659775494.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


# Explore your data

## 🔍 Exploratory Data Analysis (EDA)

Once the data is loaded and joined, the next step is **exploration**.  
This phase is not about modeling yet — it’s about **understanding what you are working with**.

### 1️⃣ Look at your columns

Start by listing and scanning all columns:
- What does each column represent?
- Which ones are numerical, categorical, dates, IDs, or text?
- Are there columns that look redundant, constant, or suspicious?

This step often reveals:
- Unexpected columns
- Naming inconsistencies
- Encoded information that needs clarification



### 2️⃣ Inspect distributions

For each column, you should ask:
- What are the typical values?
- Are there outliers or extreme values?
- Are there missing values, and how frequent are they?
- Do some columns have skewed or multi-modal distributions?

Understanding distributions helps you:
- Decide on transformations
- Detect data quality issues
- Anticipate modeling challenges



### 3️⃣ Understand what the columns *really* mean  
(scoping continues here)

Exploration is also part of the **scoping phase**.

At this stage, you are not just looking at numbers — you are trying to understand:
- How the data was generated
- Which columns are inputs vs outcomes
- Which variables are actionable, noisy, or proxy signals

Very often:
- There is no data dictionary
- Column definitions are incomplete or ambiguous
- The only way forward is to **ask questions** to someone who knows the domain

This is realistic and expected in real-world projects.



### 4️⃣ Manual vs automated exploration

You can explore the data in two complementary ways:

**Manual exploration**
- `df.head()`, `df.describe()`, `df.value_counts()`
- Custom plots and sanity checks
- Slower, but forces deep understanding

**Automated exploration tools**
- Tools like **YData Profiling** generate an HTML report with:
  - Column summaries
  - Distributions
  - Missing value analysis
  - Correlations
- Faster and great for first-pass diagnostics

Both approaches are useful:
- Automated tools give you breadth
- Manual exploration gives you depth

The goal of EDA is not to be exhaustive, but to reach a point where the data *makes sense* and the next modeling decisions are justified.


## Example with ydataprofiling

In [ ]:
from ydata_profiling import ProfileReport
import pandas as pd
df = # THE DF YOU WANT TO GENERATE A REPORT ABOUT
profile = ProfileReport(df, title="EDA Report")
profile.to_file("report.html")


/Users/othmanbensouda/Desktop/eds_workshop/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 1219.27it/s]


Ydata profiling will generate a report.html, that you can open with a browser like Google Chrome. Try it out, it's pretty cool and very fast.

# Modeling (Train/Test)

Now, you can start modeling. Ask yourself what features you need to train, what you need to test, and how you would define a good model.
Here is a quick hint and reminder regarding metrics : 

 ![precision_recall](../images/precision_recall.png)

In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score

# =========================================================
# LOAD DATA (RELATIVE PATH)
# =========================================================

DATASET_PATH = "../data/csv_files"

students = pd.read_csv(f"{DATASET_PATH}/students.csv")
attendance = pd.read_csv(f"{DATASET_PATH}/attendance_records.csv")
grades = pd.read_csv(f"{DATASET_PATH}/course_grades.csv")
labels = pd.read_csv(f"{DATASET_PATH}/dropout_labels.csv")

# =========================================================
# S1 FEATURES (TRAIN)
# =========================================================

att_s1 = attendance[attendance["academic_year"] == "2024_S1"]
grades_s1 = grades[grades["academic_year"] == "2024_S1"]

att_feat_s1 = (
    att_s1.groupby("student_id")
    .agg(
        mean_absent=("days_absent", "mean"),
        total_absent=("days_absent", "sum"),
        max_absent_week=("days_absent", "max")
    )
    .reset_index()
)

grade_feat_s1 = (
    grades_s1.groupby("student_id")
    .agg(
        mean_grade=("grade_value", "mean"),
        min_grade=("grade_value", "min")
    )
    .reset_index()
)

labels_s2 = labels[labels["semester"] == "2024_S2"][["student_id", "dropout"]]

train_df = (
    students
    .merge(att_feat_s1, on="student_id", how="inner")
    .merge(grade_feat_s1, on="student_id", how="inner")
    .merge(labels_s2, on="student_id", how="inner")
)

X_train = train_df.drop(columns=["student_id", "dropout"])
y_train = train_df["dropout"]

# =========================================================
# PREPROCESSING
# =========================================================

num_cols = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object", "bool"]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
    ]
)

# =========================================================
# MODEL
# =========================================================

model = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

pipe = Pipeline([
    ("prep", preprocessor),
    ("clf", model)
])

pipe.fit(X_train, y_train)

# =========================================================
# S2 FEATURES (TEST)
# =========================================================

s1_students = set(att_feat_s1.student_id)

att_s2 = attendance[
    (attendance["academic_year"] == "2024_S2") &
    (attendance["student_id"].isin(s1_students))
]

grades_s2 = grades[
    (grades["academic_year"] == "2024_S2") &
    (grades["student_id"].isin(s1_students))
]

att_feat_s2 = (
    att_s2.groupby("student_id")
    .agg(
        mean_absent=("days_absent", "mean"),
        total_absent=("days_absent", "sum"),
        max_absent_week=("days_absent", "max")
    )
    .reset_index()
)

grade_feat_s2 = (
    grades_s2.groupby("student_id")
    .agg(
        mean_grade=("grade_value", "mean"),
        min_grade=("grade_value", "min")
    )
    .reset_index()
)

test_df = (
    students
    .merge(att_feat_s2, on="student_id", how="inner")
    .merge(grade_feat_s2, on="student_id", how="inner")
    .merge(labels_s2, on="student_id", how="inner")
)

X_test = test_df.drop(columns=["student_id", "dropout"])
y_test = test_df["dropout"]

# =========================================================
# EVALUATION
# =========================================================

y_pred = pipe.predict(X_test)
y_prob = pipe.predict_proba(X_test)[:, 1]

print("\n=== TEST PERFORMANCE (S1 FEATURES → S2 DROPOUT) ===")
print(classification_report(y_test, y_pred))
print("ROC AUC:", round(roc_auc_score(y_test, y_prob), 3))




=== TEST PERFORMANCE (S1 FEATURES → S2 DROPOUT) ===
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4126
           1       0.74      0.70      0.72       419

    accuracy                           0.95      4545
   macro avg       0.85      0.84      0.84      4545
weighted avg       0.95      0.95      0.95      4545

ROC AUC: 0.979


## Experimenting 

In practice, model development is rarely a single run. Data scientists typically run many experiments to test different feature sets, hyperparameters, and modeling choices, while systematically tracking their results for comparison and reproducibility. 

A common tool to do this is MLflow, which also allows you to deploy your model and make it usable by your client.

 ![mlflow](../images/mlflow_experiments.png)

# Create an API

In practice, data science does not live in notebooks.

Models and decision logic are used by other systems: dashboards, internal tools, and applications.
An API is how those systems communicate with your logic. People might not ask for your code directly, but will want to interact with your model/app.

By creating an API, you:

- separate decision logic from the interface

- make your work reusable and deployable

- move from analysis to a real system

This is how models are used in production.


From the project root, run *uvicorn src.flask_app:app --reload*


If successful, you will see : Uvicorn running on http://127.0.0.1:8000

# Use your API and build a simple User interface to interact with it

An API by itself is not useful to most users.

In practice, decision-makers interact with models through simple interfaces, not code.
The role of the user interface is to collect inputs, send them to the API, and display the result in a clear and actionable way.

In this project, you will use Streamlit to build a minimal interface that communicates with your API, demonstrating how data science systems are used by real people outside notebooks.

Run in your terminal : *streamlit run src/streamlit_app.py*

# Containerize

When code runs on your laptop, it depends on:

- your operating system

- your Python version

- your installed libraries

This is why “it works on my machine” is such a common failure.

Containerization solves this by packaging:

- the code

- the dependencies

- the runtime environment

into a single, portable unit called a container.

In practice, this means:

- the API runs the same way everywhere

- deployment is predictable

- debugging environment issues disappears

This is how models are deployed in real systems.

Dockerhub is like Github for containers. There are many : Google Artifact Registry, Azure Artifact Registry etc..

![dockerhub](../images/dockerhub.png)


Once you build your dockerfile, you can build your docker image and put it on registries like dockerhub. From the project root, write *docker build -t riverbend-app .*


# Requirements file (optional)

Tip : to get a requirements.txt file with all libraries, use pip freeze > requirements.txt